
## Task 2
Create a PowerPoint presentation wihch outlines the approach we will be taking to identify which of the 1000 customers Sprocket Central Pty Ltd should target, based on this dataset. Explain the three phases:  Data Exploration; Model Development and Interpretation.


Sprocket Central Pty Ltd has given us a new list of 1000 potential customers with their demographics and attributes. However, these customers do not have prior transaction history with the organisation. 

The marketing team at Sprocket Central Pty Ltd is sure that, if correctly analysed, the data would reveal useful customer insights which could help optimise resource allocation for targeted marketing. Hence, improve performance by focusing on high value customers.



For context, Sprocket Central Pty Ltd is a long-standing KPMG client whom specialises in high-quality bikes and accessible cycling accessories to riders. Their marketing team is looking to boost business by analysing their existing customer dataset to determine customer trends and behaviour. 

Using the existing 3 datasets (Customer demographic, customer address and transactions) as a labelled dataset, please recommend which of these 1000 new customers should be targeted to drive the most value for the organisation. 

In building this recommendation, we need to start with a PowerPoint presentation which outlines the approach which we will be taking. The client has agreed on a 3 week scope with the following 3 phases as follows - Data Exploration; Model Development and Interpretation.

Prepare a detailed approach for completing the analysis including activities – i.e. understanding the data distributions, feature engineering, data transformations, modelling, results interpretation and reporting. This detailed plan needs to be presented to the client to get a sign-off. Please advise what steps you would take. 

Please ensure your PowerPoint presentation includes a detailed approach for our strategy behind each of the 3 phases including activities involved in each - i.e. understanding the data distributions, feature engineering, data transformations, modelling, results interpretation and reporting. This detailed plan needs to be presented to the client to get a sign-off.

--
Tips: Raw data fields may be transformed into other calculated fields for modelling purposes (i.e. converting D.O.B to age or age groups).  Tips: You may source external data from the ABS / Census to add additional variables that may help support your model. 

### Objectives

##### 1) Find high value customers
##### 2) Determine customer trends and behaviour
##### 3) which of these 1000 new customers should be targeted to drive the most value for the organisation
##### 4) Data Exploration; Model Development and Interpretation- PPT on the project approach(understanding the data distributions, feature engineering, data transformations, modelling, results interpretation and reporting)

##### can customers be clustered based on their value?


In [1]:
import pandas as pd 
from matplotlib import pyplot as plt 
import seaborn as sns 
import matplotlib 

In [2]:
excelFile = pd.ExcelFile("newData.xlsx")       

In [3]:
Transactions = pd.read_excel(excelFile, 'Transactions', skiprows=[0]) 
CustomerDemographic = pd.read_excel(excelFile,'CustomerDemographic', skiprows=[0]) 
CustomerAddress = pd.read_excel(excelFile, 'CustomerAddress', skiprows=[0])
newCustomers = pd.read_excel(excelFile,"NewCustomerList", skiprows = [0])
pd.set_option("display.max_columns",100)
pd.set_option("display.max_rows",None) 

In [4]:
data = pd.merge(CustomerDemographic,CustomerAddress, on="customer_id")  
data = pd.merge(Transactions,data, on="customer_id") 
data.to_csv("customerData.csv") 

### Task2
#### list_price : The list price, also known as the manufacturer's suggested retail price (MSRP), or the recommended retail price (RRP)
#### standard_cost :Standard costs are estimates of the actual costs in a company's production process, because actual costs cannot be known in advance.

#### clustering customers by the buisness value they generate

In [21]:
data.head(1)

,transaction_id,product_id,customer_id,transaction_date,online_order,order_status,brand,product_line,product_class,product_size,list_price,standard_cost,product_first_sold_date,first_name,last_name,gender,past_3_years_bike_related_purchases,DOB,job_title,job_industry_category,wealth_segment,deceased_indicator,default,owns_car,tenure,address,postcode,state,country,property_valuation
0,1,2,2950,2017-02-25,0.0,Approved,Solex,Standard,medium,medium,71.49,53.62,41245.0,Kristos,Anthony,Male,19,1955-01-11,Software Engineer I,Financial Services,Mass Customer,N,ã»(ï¿£âï¿£)ã»:*:,Yes,10.0,984 Hoepker Court,3064,VIC,Australia,6


In [55]:
custFreq = data.loc[:,"customer_id"].value_counts()         # [2476]
data["custFreq"] = 0